In [1]:
import torch
from torch import nn
from torch.autograd import Variable

## Input:

1. ieas_results → binary array `n=23`


2. model_path → .pth file containing the weights of the trained model



In [2]:
def create_model():
    
    D_in = 4
    H1 = 100
    H2 = 75
    H3 = 50
    H4 = 25
    D_out = 13
    
    return nn.Sequential(
        nn.Linear(D_in, H1),
        nn.Tanh(),
        nn.Linear(H1, H2),
        nn.ReLU(),
        nn.Linear(H2, H3),
        nn.ReLU(),
        nn.Linear(H3,H4),
        nn.ReLU(),
        nn.Linear(H4, D_out),
        nn.Softmax(dim=0)
    )



In [3]:
def create_feature_vector(responses):
    if len(responses) != 23:
        print("The survey has missing values.")
        return 0
    else:
        feature_vector = Variable(torch.Tensor([sum(responses[0:6])/6 , sum(responses[7:14])/8,
                          sum(1-responses[15:20])/6 , sum(1-responses[21:23])/3]))
        return feature_vector



In [4]:
def bpmll(ieas_results,model_path):
    
    model = create_model()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    feature_vector = create_feature_vector(Variable(torch.Tensor(ieas_results)))
    output = model(feature_vector)
    categories = torch.argsort(output).tolist()[::-1]
    categories_sorted = [categories[i] + 1 for i in range(len(categories))]
    
    return(categories_sorted)



## Output:

1. categories_sorted → array `n=13`
    * contains the categories (i.e. `int` values from 1 to 13)
    * sorted by highest probability
    * example: `[10, 6, 3, 9, 12, 1, 2, 8, 13, 4, 5, 7, 11]`
    * **category 10** is the most likely to help while **category 11** is least likely
    * 1 indexed to match semantic naming convention of task JSONs and NumPy resources to be used in the next step (NLP)